# TODO: Come up with a title

Music has been an integral part of world cultures for thousands of years. Styles and songs have come and gone, but the concept of rating music has been a relatively new concept. Since the late 1950s, Billboard has been tracking the popularity of songs....

TODO: Finish this intro!

In [ ]:
# Imports for the project

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read in the data frame to start our analysis

df1 = pd.read_json('./full_data.json')
df2 = pd.read_json('./full_data.json')

In [ ]:
# Now, let's sort this data to get it formatted where each row is a full chart and each entry is a
# list of attributes relating to the song at that position

sort_og_df = df1.reindex_axis(sorted(df1.columns), axis=1)
sort_mod_df = df2.reindex_axis(sorted(df2.columns), axis=1)

# Let's examine the dataframe
sort_og_df.head()


We have our song data! Now to extract and clean it up a bit

In [ ]:
# Indices:
# ChartWeek, SongName, Artist, PeakPos, LastPos, #Weeks

We'll want to change the columns to represent a specific attribute of the chart. Currently, Each chart takes up a full row where each columns is the particular rank of a song in that chart. We want each column to read (in order): Week of the chart, name of the song, artist, highest position on the chart, the song's position from the week prior, and the number of weeks on the chart

In [ ]:
# Save the current chart week numbers in a list for restoration later

SongWeeks = []
for num in range(len(sort_mod_df)):
    
    SongWeeks.append(num)

In [ ]:
# Next, put all of the song attributes into separate lists to use as columns

# Create the lists
SongName = []
Artist = []
PeakPos = []
LastPos = []
NumWeeks = []

# Iterate through the entire dataframe
for row, col in sort_mod_df.iterrows():
    
    # Grab particular entries from each column in the current row
    for ent in col:
        
        SongName.append(ent[0])
        Artist.append(ent[1])
        PeakPos.append(ent[2])
        LastPos.append(ent[3])
        NumWeeks.append(ent[4])

In [ ]:
# Clear out the copy dataframe to rebuild with new lists

for num in range(len(SongWeeks)):
    
    sort_mod_df.drop(num, inplace=True)

In [ ]:
# Now, rebuild the dataframe using the lists as the columns

Now that we have our nicely sorted data, let's start analyzing it to see what trends we can find